The goal of this notebook is, given a metric, to compute Christoffel symbols, Riemann curvature tensor, Ricci tensor, Ricci scalar and Eintein tensor. Supported metrics include 2D spherical, 2D toro, Schwarzschild, Wormhole, Alcubierre

In [3]:
import sympy
import numpy as np
from sympy import sin, cos, Function

sympy.init_printing()  # for pretty printing

**2D Spherical**

In [4]:
syms = sympy.symbols("theta phi")
theta, phi = syms
R = sympy.symbols("R")
m = sympy.Array(sympy.diag(R**2, R**2 * sin(theta)**2))
m


⎡ 2            ⎤
⎢R       0     ⎥
⎢              ⎥
⎢     2    2   ⎥
⎣0   R ⋅sin (θ)⎦

**2D toro**

In [ ]:
syms = sympy.symbols("r phi")
r, phi = syms
R = sympy.symbols("R")
m = sympy.Array(sympy.diag((R + r*cos(phi))**2, r**2))
m

⎡              2    ⎤
⎢(R + r⋅cos(φ))   0 ⎥
⎢                   ⎥
⎢                  2⎥
⎣       0         r ⎦

**Schwarzschild**

In [ ]:
syms = sympy.symbols("t r theta phi")
t, r, theta, phi = syms
M = sympy.symbols("M")
gtt = -(1-(2*M/r))
grr = 1/ (-gtt)
m = sympy.Array(sympy.diag(gtt, grr, r**2, r**2 * sin(theta)**2))
m

⎡2⋅M                               ⎤
⎢─── - 1      0      0       0     ⎥
⎢ r                                ⎥
⎢                                  ⎥
⎢             1                    ⎥
⎢   0     ─────────  0       0     ⎥
⎢           2⋅M                    ⎥
⎢         - ─── + 1                ⎥
⎢            r                     ⎥
⎢                                  ⎥
⎢                     2            ⎥
⎢   0         0      r       0     ⎥
⎢                                  ⎥
⎢                         2    2   ⎥
⎣   0         0      0   r ⋅sin (θ)⎦

**Wormhole**

In [ ]:
syms = sympy.symbols("t r theta phi")
t, r, theta, phi = syms
b = sympy.symbols("b")
gtt = (b**2 + r**2)
gpp = gtt * sin(theta)**2
m = sympy.Array(sympy.diag(-1, 1, gtt, gpp))
m

⎡-1  0     0             0        ⎤
⎢                                 ⎥
⎢0   1     0             0        ⎥
⎢                                 ⎥
⎢        2    2                   ⎥
⎢0   0  b  + r           0        ⎥
⎢                                 ⎥
⎢                ⎛ 2    2⎞    2   ⎥
⎣0   0     0     ⎝b  + r ⎠⋅sin (θ)⎦

**Alcubierre**

In [ ]:
syms = sympy.symbols("t x y z")
t, x, y, z = syms
R = sympy.symbols("R")
x_s = sympy.symbols("x_s", cls=Function)
r_s = (x - x_s(t)) ** 2 + y ** 2 + z ** 2
f_s = 1 - (r_s/R)**4
M = sympy.diag(x_s(t).diff(t) ** 2 * f_s ** 2-1, 1, 1, 1)
M[0,1] = -2 * x_s(t).diff(t) * f_s
M[1,0] = M[0,1]
m = sympy.Array(M.tolist())
m 

⎡                               2                                             
⎢⎛                            4⎞                       ⎛                      
⎢⎜    ⎛ 2    2              2⎞ ⎟             2         ⎜    ⎛ 2    2          
⎢⎜    ⎝y  + z  + (x - xₛ(t)) ⎠ ⎟  ⎛d        ⎞          ⎜    ⎝y  + z  + (x - xₛ
⎢⎜1 - ─────────────────────────⎟ ⋅⎜──(xₛ(t))⎟  - 1  -2⋅⎜1 - ──────────────────
⎢⎜                 4           ⎟  ⎝dt       ⎠          ⎜                 4    
⎢⎝                R            ⎠                       ⎝                R     
⎢                                                                             
⎢     ⎛                            4⎞                                         
⎢     ⎜    ⎛ 2    2              2⎞ ⎟                                         
⎢     ⎜    ⎝y  + z  + (x - xₛ(t)) ⎠ ⎟ d                                       
⎢  -2⋅⎜1 - ─────────────────────────⎟⋅──(xₛ(t))                          1    
⎢     ⎜                 4           ⎟ dt            

Compute inverse of metric:

In [5]:
#m_inv = sympy.Array(M.inv()) # uncomment for Alcubierre in particular, non-diagonal metrics in general
m_inv = m.applyfunc(lambda x: 0 if x == 0 else 1/x)
m_inv

⎡1             ⎤
⎢──      0     ⎥
⎢ 2            ⎥
⎢R             ⎥
⎢              ⎥
⎢        1     ⎥
⎢0   ──────────⎥
⎢     2    2   ⎥
⎣    R ⋅sin (θ)⎦

Compute [**Christoffel symbols**](https://hirata10.github.io/ph6820/lec08_christoffel.pdf) (Lecture 8, section VI eq 41) with all three down indices:

In [7]:
dim = m.shape[0]
ch_d = sympy.MutableDenseNDimArray(range(dim*dim*dim), (dim,dim,dim))*0
for i in range(ch_d.shape[0]):
  for j in range(ch_d.shape[1]):
    for k in range(ch_d.shape[2]):
      ch_d[i,j,k] = 0.5 * (m[j,i].diff(syms[k]) + m[i,k].diff(syms[j]) - m[k,j].diff(syms[i]))
ch_d

⎡⎡0            0          ⎤  ⎡                           2              ⎤⎤
⎢⎢                        ⎥  ⎢         0            1.0⋅R ⋅sin(θ)⋅cos(θ)⎥⎥
⎢⎢         2              ⎥  ⎢                                          ⎥⎥
⎢⎣0  -1.0⋅R ⋅sin(θ)⋅cos(θ)⎦  ⎢     2                                    ⎥⎥
⎣                            ⎣1.0⋅R ⋅sin(θ)⋅cos(θ)           0          ⎦⎦

Raise first index via metric multiplication (tensorproduct followed by contration on the selected indices - one up from metric, first down from Christoffel symbol-):

In [8]:
temp = sympy.tensorproduct(m_inv, ch_d)
#ch = temp[:,0,0,:,:]  + temp[:,1,1,:,:]
ch = sympy.tensorcontraction(temp,(1,2))
#ch = sum([sympy.simplify(temp[:,i,i,:,:]) for i in range(dim)],sympy.MutableDenseNDimArray(range(dim*dim*dim), (dim,dim,dim))*0)
ch

⎡                         ⎡            1.0⋅cos(θ)⎤⎤
⎢                         ⎢    0       ──────────⎥⎥
⎢⎡0          0         ⎤  ⎢              sin(θ)  ⎥⎥
⎢⎢                     ⎥  ⎢                      ⎥⎥
⎢⎣0  -1.0⋅sin(θ)⋅cos(θ)⎦  ⎢1.0⋅cos(θ)            ⎥⎥
⎢                         ⎢──────────      0     ⎥⎥
⎣                         ⎣  sin(θ)              ⎦⎦

Compute the **[Riemann](https://hirata10.github.io/ph6820/lec10_curvature.pdf) curvature tensor** (Lecture 10, section III, eq 7):

In [9]:
rie = sympy.MutableDenseNDimArray(range(dim*dim*dim*dim), (dim,dim,dim,dim))*0
for i in range(rie.shape[0]):
  for j in range(rie.shape[1]):
    for k in range(rie.shape[2]):
      for l in range(rie.shape[3]):
        sum_1 = sum([ch[i,m,k]*ch[m,j,l] for m in range(dim)])
        sum_2 = sum([ch[i,m,l]*ch[m,j,k] for m in range(dim)])
        rie[i,j,k,l] = sympy.simplify(ch[i,j,l].diff(syms[k]) - ch[i,j,k].diff(syms[l]) + sum_1 - sum_2)
rie

⎡             ⎡                     2   ⎤⎤
⎢  ⎡0  0⎤     ⎢     0        1.0⋅sin (θ)⎥⎥
⎢  ⎢    ⎥     ⎢                         ⎥⎥
⎢  ⎣0  0⎦     ⎢        2                ⎥⎥
⎢             ⎣-1.0⋅sin (θ)       0     ⎦⎥
⎢                                        ⎥
⎢⎡ 0   -1.0⎤            ⎡0  0⎤           ⎥
⎢⎢         ⎥            ⎢    ⎥           ⎥
⎣⎣1.0   0  ⎦            ⎣0  0⎦           ⎦

Riemann tensor can also be computed taking [simmetries](https://hirata10.github.io/ph6820/lec10_curvature.pdf) into account (Lecture 10, Section III.A):

In [11]:
rie = sympy.MutableDenseNDimArray(range(dim*dim*dim*dim), (dim,dim,dim,dim))*0
for i in range(rie.shape[0]):
  for j in range(rie.shape[1]):
    for k in range(rie.shape[2]):
      for l in range(rie.shape[3]):
        if k > l:
          rie[i,j,k,l] = -rie[i,j,l,k]
        else:
          sum_1 = sum([ch[i,m,k]*ch[m,j,l] for m in range(dim)])
          sum_2 = sum([ch[i,m,l]*ch[m,j,k] for m in range(dim)])
          rie[i,j,k,l] = sympy.simplify(ch[i,j,l].diff(syms[k]) - ch[i,j,k].diff(syms[l]) + sum_1 - sum_2)
rie

⎡             ⎡                     2   ⎤⎤
⎢  ⎡0  0⎤     ⎢     0        1.0⋅sin (θ)⎥⎥
⎢  ⎢    ⎥     ⎢                         ⎥⎥
⎢  ⎣0  0⎦     ⎢        2                ⎥⎥
⎢             ⎣-1.0⋅sin (θ)       0     ⎦⎥
⎢                                        ⎥
⎢⎡ 0   -1.0⎤            ⎡0  0⎤           ⎥
⎢⎢         ⎥            ⎢    ⎥           ⎥
⎣⎣1.0   0  ⎦            ⎣0  0⎦           ⎦

Simmetries can be observed when all indices of the Riemnan tensor are down, so we bring the first index down via metric multiplication:

In [13]:
sympy.tensorcontraction(sympy.tensorproduct(m,rie),(1,2))

⎡                                   ⎡                      2    2   ⎤⎤
⎢             ⎡0  0⎤                ⎢       0         1.0⋅R ⋅sin (θ)⎥⎥
⎢             ⎢    ⎥                ⎢                               ⎥⎥
⎢             ⎣0  0⎦                ⎢      2    2                   ⎥⎥
⎢                                   ⎣-1.0⋅R ⋅sin (θ)        0       ⎦⎥
⎢                                                                    ⎥
⎢⎡                      2    2   ⎤                                   ⎥
⎢⎢      0         -1.0⋅R ⋅sin (θ)⎥               ⎡0  0⎤              ⎥
⎢⎢                               ⎥               ⎢    ⎥              ⎥
⎢⎢     2    2                    ⎥               ⎣0  0⎦              ⎥
⎣⎣1.0⋅R ⋅sin (θ)         0       ⎦                                   ⎦

Now compute the **Ricci tensor** by contracting the Rieman tensor:

In [14]:
#ric = rie[0,:,0,:] + rie[1,:,1,:]
ric = sympy.tensorcontraction(rie,(0,2))
#ric = sum([rie[i,:,i,:] for i in range(dim)],sympy.MutableDenseNDimArray(range(dim*dim), (dim,dim))*0)
ric

⎡1.0       0     ⎤
⎢                ⎥
⎢            2   ⎥
⎣ 0   1.0⋅sin (θ)⎦

Note that due to [symmetries](https://hirata10.github.io/ph6820/lec11_efe.pdf) (Lecture 11, section III, eq 1), we could have chosen different down indices and get either the analogous 
result, or zeros:

In [15]:
#rie[0,:,:,0] + rie[1,:,:,1]
sympy.tensorcontraction(rie,(0,3))
#sum([rie[i,:,:,i] for i in range(dim)],sympy.MutableDenseNDimArray(range(dim*dim), (dim,dim))*0)


⎡-1.0       0      ⎤
⎢                  ⎥
⎢              2   ⎥
⎣ 0    -1.0⋅sin (θ)⎦

and the other option would be all zeros:

In [16]:
#rie[0,0,:,:] + rie[1,1,:,:]
sympy.tensorcontraction(rie,(0,1))
#sum([rie[i,i,:,:] for i in range(dim)],sympy.MutableDenseNDimArray(range(dim*dim), (dim,dim))*0)


⎡0  0⎤
⎢    ⎥
⎣0  0⎦

The Ricci tensor now has two down indices. To compute the Ricci scalar, first raise one index in the Ricci tensor, (via metric multiplication: tensor product followed by contraction):

In [17]:
temp = sympy.tensorproduct(m_inv,ric)
#ric2 = temp[0,:,0,:] + temp[1,:,1,:]
ric2 = sympy.tensorcontraction(temp,(0,2))
#ric2 = sum([temp[i,:,i,:] for i in range(dim)],sympy.MutableDenseNDimArray(range(dim*dim), (dim,dim))*0)
ric2

⎡1.0     ⎤
⎢───   0 ⎥
⎢  2     ⎥
⎢ R      ⎥
⎢        ⎥
⎢     1.0⎥
⎢ 0   ───⎥
⎢       2⎥
⎣      R ⎦

And finally contract the Ricci tensor to get the **Ricci scalar**:

In [18]:
#ric_s = ric2[0,0] + ric2[1,1]
ric_s = sympy.tensorcontraction(ric2,(0,1))
#ric_s = sum([ric2[i,i] for i in range(dim)])
ric_s

[**Einstein Tensor**](https://hirata10.github.io/ph6820/lec11_efe.pdf) (Lecture 11, section IV.A, equation 15), with down indices, is then:

In [19]:
ein = ric - 0.5 * ric_s * m
ein

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

(which is 0 as expected for 2D spherical metric, given that it corresponds to flat spacetime)